In [15]:
import networkx as nx
import numpy as np
import d3networkx as d3nx

In [23]:
G = d3nx.d3graph.D3Graph(nx.read_weighted_edgelist("data/path.edgelist"))

In [54]:
node_pos = {
    "A": (0, 1),
    "S": (1, 0),
    "B": (1, 3),
    "D": (0, 4),
    "H": (1, 5),
    "F": (0, 6),
    "G": (2, 6),
    "C": (3, 0),
    "I": (3, 4),
    "E": (3, 6),
    "L": (4, 2),
    "K": (4, 4),
    "J": (5, 3),
}

In [27]:
d3 = await d3nx.create_d3nx_visualizer()
d3.clear()
d3.set_graph(G)
d3.update()

websocket server started...

networkx connected...visualizer connected...

In [51]:
import networkx as nx
import heapq
from asyncio import sleep
from typing import Union, Hashable
import more_itertools

Node = Hashable


def dijkstra_search(graph: Union[nx.Graph, nx.DiGraph], source: Node, target: Node):
    """
    Find the shortest path between two nodes in a weighted graph using Dijkstra's algorithm.

    Args:
        graph: The input graph represented as a NetworkX Graph object.
        source: The index of the source node.
        target: The index of the target node.

    Returns:
        A tuple containing the shortest path as a list of node IDs and its total length.
    """
    # Determine if the graph is directed or undirected
    is_directed = isinstance(graph, nx.DiGraph)

    # Initialize the distances of all nodes to infinity
    distance: dict[Node, float] = {node: float("inf") for node in graph.nodes()}
    distance[source] = 0

    # Initialize the heap queue with the source node
    heap = [(0, source)]

    # Initialize the predecessor dictionary
    predecessors = {}

    while heap:
        # Pop the node with the minimum distance from the heap queue
        curr_dist, curr_node = heapq.heappop(heap)

        # If we've reached the target node, terminate early
        if curr_node == target:
            break

        # Iterate over the neighbors of the current node
        neighbors = (
            graph.successors(curr_node) if is_directed else graph.neighbors(curr_node)
        )
        for neighbor in neighbors:
            # Calculate the distance from the current node to the neighbor
            edge_weight = graph[curr_node][neighbor]["weight"]
            neighbor_dist = curr_dist + edge_weight

            # Update the distance and predecessor of the neighbor if a shorter path was found
            if neighbor_dist < distance[neighbor]:
                distance[neighbor] = neighbor_dist
                predecessors[neighbor] = curr_node

                # Add the neighbor to the heap queue
                heapq.heappush(heap, (neighbor_dist, neighbor))

        yield predecessors, distance


def predecessor_path(predecessors: dict, source: Node, target: Node):
    # Construct the shortest path from the predecessor dictionary
    path = [target]
    while path[-1] != source:
        path.append(predecessors[path[-1]])
        yield path

    return path


async def dijkstra_d3(
    d3: d3nx.D3NetworkxRenderer,
    graph: Union[nx.Graph, nx.DiGraph],
    source: Node,
    target: Node,
    sleep_time=0.5,
):
    d3.clear_highlights()
    d3.highlight_nodes((source,))
    d3.update()

    predecessors = {}
    distances = {}
    for predecessors_, distances_ in dijkstra_search(graph, source, target):
        predecessors, distances = (predecessors_, distances_)
        d3.highlight_nodes(predecessors_.keys())
        d3.update()
        await sleep(sleep_time)

    d3.clear_highlights()
    d3.update()
    await sleep(sleep_time)

    final_path = []
    for path in predecessor_path(predecessors, source, target):
        final_path = path
        d3.highlight_nodes(path)
        d3.highlight_edges(more_itertools.pairwise(path))
        d3.update()
        await sleep(sleep_time)

    final_path.reverse()

    return final_path, distances[target]

In [53]:
await dijkstra_d3(d3, G, "S", "E")

(['S', 'B', 'H', 'G', 'E'], 7.0)

In [44]:
d3.clear_highlights()
d3.update()

In [50]:
class KeepValue:
    """Based on: https://stackoverflow.com/a/34073559"""
    def __init__(self, gen):
        self.gen = gen

    def __iter__(self):
        self.value = yield from self.gen


def foo():
    for i in range(5):
        yield i * 2
    return "done!"

gen = foo()
for x in gen:
    print(x)
print(gen.value)

0
2
4
6
8
done!
